# MNIST Classification Problem

In [2]:
# Imports
import os
import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Data

In [3]:
# Extracting the data
data_path = os.getcwd() + "/data/train.csv"
data = pd.read_csv(data_path)
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Converting data to NumPy array and suffling it to ensure randomness
data = np.array(data)
np.random.shuffle(data)

# Dimensions of the data
m, n = data.shape
print(m, n)

# Extracting a portion of the dataset for cross-validation
data_cv = data[0:1000].T
Y_cv = data_cv[0] # Labels
X_cv = data_cv[1:n]
X_cv = X_cv / 255.

# The rest of the dataset is saved as training data
data_t = data[1000:m].T
Y_t = data_t[0]
X_t = data_t[1:n]
X_t = X_t / 255

42000 785


In [5]:
Y_t, X_t.shape

(array([8, 4, 4, ..., 2, 1, 8]), (784, 41000))

### Model 

In [6]:
def init_model():
    # Random weights and biases between -0.5 and 0.5
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

# Returns 0 if Z_i < 0 
def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    result = np.exp(Z) / sum(np.exp(Z))
    return result

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    a1 = ReLU(Z1)
    Z2 = W2.dot(a1) + b2
    a2 = softmax(Z2)
    return Z1, a1, Z2, a2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, a1, Z2, a2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = a2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(a1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    # The alpha parameter defines the size of the weight and bias correction for each iteration.
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2 
    return W1, b1, W2, b2

In [7]:
def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_model()
    for i in range(iterations + 1):
        Z1, a1, Z2, a2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, a1, Z2, a2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(a2), Y))
    return W1, b1, W2, b2 

In [8]:
W1, b1, W2, b2 = gradient_descent(X_t, Y_t, 0.15, 500)

iteration:  0
[4 4 4 ... 3 7 1] [8 4 4 ... 2 1 8]
Accuracy:  0.10953658536585366
iteration:  10
[4 4 4 ... 8 1 1] [8 4 4 ... 2 1 8]
Accuracy:  0.2380731707317073
iteration:  20
[8 8 8 ... 8 1 1] [8 4 4 ... 2 1 8]
Accuracy:  0.33446341463414636
iteration:  30
[8 8 8 ... 8 1 1] [8 4 4 ... 2 1 8]
Accuracy:  0.3972926829268293
iteration:  40
[9 8 8 ... 8 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.45446341463414636
iteration:  50
[9 8 8 ... 8 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.49553658536585365
iteration:  60
[9 4 8 ... 8 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.5337560975609756
iteration:  70
[9 4 8 ... 8 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.5716829268292682
iteration:  80
[9 4 8 ... 6 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.6069756097560975
iteration:  90
[9 4 8 ... 6 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.6370731707317073
iteration:  100
[9 4 8 ... 6 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.6619756097560976
iteration:  110
[9 4 8 ... 6 1 8] [8 4 4 ... 2 1 8]
Accuracy:  0.6836341463414635
iteration:  120
[9 4 4 